In [1]:
!pip3 install -q -U git+https://github.com/huggingface/transformers.git

!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/
!pip3 install -q -U git+https://github.com/huggingface/optimum.git

!pip3 install -q -U flash-attn
!pip install -q -U bitsandbytes
!pip install -q -U accelerate


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 97.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 95.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextStreamer
import torch
import re
import json
import pandas as pd
import random

Model

In [3]:
model_id = "TheBloke/CodeLlama-7B-Instruct-GPTQ"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
          bnb_4bit_use_double_quant=True,
          bnb_4bit_quant_type="nf4",
          bnb_4bit_compute_dtype=torch.bfloat16
 )

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map='auto',
    torch_dtype=torch.float16,
    revision = "main",
    trust_remote_code = True,
    cache_dir = ''
    )
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

configuration_llama.py:   0%|          | 0.00/8.56k [00:00<?, ?B/s]

modeling_llama.py:   0%|          | 0.00/45.9k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [24]:
# Pipeline

# Delimiters
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

system_prompt="""You are a helpful assistant."""

# Define a stream *without* function calling capabilities
def stream(user_prompt):
    prompt = f"{B_INST} {B_SYS}{system_prompt.strip()}{E_SYS}{user_prompt.strip()} {E_INST}"

    inputs = tokenizer([prompt], return_tensors="pt").input_ids.cuda()
    streamer = TextStreamer(tokenizer)

    with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=False, enable_mem_efficient=False):
        generated_text = model.generate(inputs=inputs, streamer=streamer, max_new_tokens=70, temperature=0.01, do_sample=True)

    # Decode the generated text
    generated_text = tokenizer.decode(generated_text[0], skip_special_tokens=True)

    return generated_text

Zero-prompt

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
with open('/content/drive/MyDrive/comments_for_prompts.json') as f:
    reddit_comments = json.load(f)

with open('/content/drive/MyDrive/anes_questions.json') as q:
    questions = json.load(q)

anes = pd.read_csv('/content/drive/MyDrive/anes_timeseries_2020_csv_20220210.csv')
# Question had an option for free text answer, which is not useful for the analysis
# turned into negative value to not take it into account
anes['V201336'].replace(5, -7, inplace=True)

questcode_topic = {"V201336": "abortion1",
              "V201342x": "abortion2",
              "V202332": "climate change1",
              "V202333": "climate change2",
              "V202337": "gun control1",
              "V202341x": "gun control2",
              "V202344x": "gun control3",
              "V202232": "immigration1",
              "V202233": "immigration2",
              "V202236x":"immigration3",
              "V202380x": "healthcare1"}

<ipython-input-8-97e4e7f0e94f>:7: DtypeWarning: Columns (15,17,18,19,21,22,23,25,26,27,29,30,31,33,34,35,37,38,1508,1509) have mixed types. Specify dtype option on import or set low_memory=False.
  anes = pd.read_csv('/content/drive/MyDrive/anes_timeseries_2020_csv_20220210.csv')


In [9]:
def prompt_zero(question, num_respondents):
    prompt_text = 'In the following you will be given a question followed several possible answer categories. Please \
    estimate how a representative sample of %d US-Americans would have answered the question in 2020 by providing the\
    number of respondents for each answer option in percent (output a dictionary object with answer category number as\
    key and answer frequency in percent as value. Only provide the number without the percentage sign).\
    Here is the questions: %s. '%(num_respondents, question)

    return prompt_text

# without imputing number of responsents - sometimes confusing for model
def prompt_zero1(question):
    prompt_text = 'In the following you will be given a question followed several possible answer categories. Please \
    estimate how a representative sample of US-Americans would have answered the question in 2020 by providing the\
    number of respondents for each answer option in percent (output a dictionary object with answer category number as\
    key and answer frequency in percent as value. Only provide the number without the percentage sign).\
    Here is the questions: %s. '%(question)

    return prompt_text

In [25]:
cllama_zero = {}
runs = 1
num_comments = 100

# Run the stream twice
for i in range(runs):
    for question in questions:
        # Call the stream function to get the answer
        answer = stream(prompt_zero1(question))

        # Extract the answer dictionary from the response
        answer_clean = re.findall(r'\{([^{}]*)\}', answer)
        answer_dicts = []
        for item in answer_clean:
            # Split by comma or newline
            items_split = re.split(r',|\n', item)
            # Create dictionary from split items
            pairs = [pair.split(':') for pair in items_split]
            # Filter out invalid pairs
            valid_pairs = [pair for pair in pairs if len(pair) == 2]
            # Create dictionary from valid pairs
            answer_dict = dict(valid_pairs)
            answer_dicts.append(answer_dict)
        # Add the answer dictionary to the list of answers for the current question
        if question not in cllama_zero:
            cllama_zero[question] = []
        cllama_zero[question].extend(answer_dicts)

        torch.cuda.empty_cache()

<s> [INST] <<SYS>>
You are a helpful assistant.
<</SYS>>

In the following you will be given a question followed several possible answer categories. Please     estimate how a representative sample of US-Americans would have answered the question in 2020 by providing the    number of respondents for each answer option in percent (output a dictionary object with answer category number as    key and answer frequency in percent as value. Only provide the number without the percentage sign).    Here is the questions: V201336. [/INST]  I'm happy to help! However, I want to clarify that the question you provided is not a question that can be answered in 2020. It is a question that was asked in 2013, and it is not appropriate to provide an answer for it.

If you have any other questions or concerns
<s> [INST] <<SYS>>
You are a helpful assistant.
<</SYS>>

In the following you will be given a question followed several possible answer categories. Please     estimate how a representative sample o

KeyboardInterrupt: 

In [12]:
cllama_zero

# cllama_answers = json.dumps(cllama_zero)
# with open("cllama_zero.json", "w") as file:
#     file.write(cllama_answers)

{'V201336': [],
 'V201342x': [],
 'V202332': [],
 'V202333': [],
 'V202337': [],
 'V202341x': [],
 'V202344x': [],
 'V202232': [],
 'V202233': [],
 'V202236x': [],
 'V202380x': []}

Few-shot prompt

In [13]:
def prompt_comments(topic, discussion, question, num_respondents, num_comments = 100):
    headline = discussion[0][10:-1]

    shuffled_comments = discussion[1:]
    random.shuffle(shuffled_comments)
    comments = shuffled_comments[:num_comments]

    prompt_text = 'This is discussion on %s was held on Reddit in 2020. They are comments to a news article named "%s". \
    The comments score represents the net value of upvotes and downvotes. Afterwards you will be given a question followed \
    several possible answer categories. \
    Comments: %s.   \
    Please estimate how a representative sample of %d US-Americans would have answered the following question in 2020. Provide the \
    number of respondents for each answer option in percent (output a dictionary object with answer category number as \
    key and answer frequency in percent as value. Only provide the number without the percentage sign). \
    Here are the question and categories: %s. '%(topic, headline, comments, num_respondents, question)

    return prompt_text

def prompt_comments_strictly(topic, discussion, question, num_respondents, num_comments = 100):
    headline = discussion[0][10:-1]

    shuffled_comments = discussion[1:]
    random.shuffle(shuffled_comments)
    comments = shuffled_comments[:num_comments]

    prompt_text = 'This is discussion on %s was held on Reddit in 2020. They are comments to a news article named "%s". \
    The comments score represents the net value of upvotes and downvotes. Afterwards you will be given a question followed \
    several possible answer categories.\
    Comments: %s.   \
    Please estimate how a representative sample of %d US-Americans would have answered the following question in 2020.\
    Base yourself strictly in the comments provided. Output the number of respondents for each answer option in percent\
    (output a dictionary object with answer category number as key and answer frequency in percent as value.\
    Only provide the number without the percentage sign).\
    Here are the question and categories: %s'%(topic, headline, comments, num_respondents, question)

    return prompt_text

In [26]:
cllama_few = {}
runs = 1
num_comments = 100

# Run the stream twice
for i in range(runs):
    for question in questions:

        valid_ans = anes[anes[question] > 0]
        survey_ans = len(valid_ans)
        topic = questcode_topic[question][:-1]

        # Call the stream function to get the answer
        answer = stream(prompt_comments_strictly(topic, reddit_comments[topic], questions[question], survey_ans, num_comments))

        # Extract the answer dictionary from the response
        answer_clean = re.findall(r'\{([^{}]*)\}', answer)
        answer_dicts = []
        for item in answer_clean:
            # Split by comma or newline
            items_split = re.split(r',|\n', item)
            # Create dictionary from split items
            pairs = [pair.split(':') for pair in items_split]
            # Filter out invalid pairs
            valid_pairs = [pair for pair in pairs if len(pair) == 2]
            # Create dictionary from valid pairs
            answer_dict = dict(valid_pairs)
            answer_dicts.append(answer_dict)
        # Add the answer dictionary to the list of answers for the current question
        if question not in cllama_few:
            cllama_few[question] = []
        cllama_few[question].extend(answer_dicts)

        torch.cuda.empty_cache()

<s> [INST] <<SYS>>
You are a helpful assistant.
<</SYS>>

This is discussion on abortion was held on Reddit in 2020. They are comments to a news article named "Group buys Alabama abortion clinic to keep it from closing".     The comments score represents the net value of upvotes and downvotes. Afterwards you will be given a question followed     several possible answer categories.    Comments: ["Comment 94: 'Lol make everyone who chipped in watch an abortion in person.  Or better yet, let them rip a limb off of a baby.' (score: -8)\n", "Comment 93: 'Reddit hates children.' (score: -8)\n", "Comment 2: 'Finally, some good news. I think it's every woman's right to have safe abortions.' (score: 259)\n", "Comment 22: 'If there's any state that needs more abortion clinics, it's Alabama.' (score: 11)\n", "Comment 120: 'That’s promoting murder right there... smh' (score: -19)\n", "Comment 106: 'God help us all. Jesus please. Restore. Heal. Make righteous. Thank you for everything you do, your 

KeyboardInterrupt: 

In [23]:
cllama_few

{'V201336': [],
 'V201342x': [],
 'V202332': [{'"1. Not at all"': ' 23.8%',
   '"2. A little"': ' 34.5%',
   '"3. A moderate amount"': ' 29.7%',
   '"4. A lot"': ' 13.9%',
   '"5. A great deal"': ' 1.5%'}],
 'V202333': [],
 'V202337': [{'"1"': ' 37.5%', '"2"': ' 37.5%', '"3"': ' 25%'}],
 'V202341x': [{'"1"': ' 37.5%',
   '"2"': ' 22.5%',
   '"3"': ' 15%',
   '"4"': ' 15%',
   '"5"': ' 10%',
   '"6"': ' 5%',
   '"7"': ' 5%'}],
 'V202344x': [{'"Favor a great deal"': ' 118',
   '"Favor a moderate amount"': ' 1',
   '"Favor a little"': ' 1',
   '"Neither favor nor oppose"': ' 1',
   '"Oppose a little"': ' 1',
   '"Oppose a moderate amount"': ' 1',
   '"Oppose a great deal"': ' 1'}],
 'V202232': [],
 'V202233': [{'1': ' 12', ' 2': ' 25', ' 3': ' 20', ' 4': ' 45'}],
 'V202236x': [],
 'V202380x': []}